In [ ]:
from keras.models import load_model
import cv2
import numpy as np
from tensorflow.keras.optimizers import RMSprop
import os
from pathlib import Path
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
model = load_model('model.h5')
model.compile(optimizer=RMSprop(learning_rate=0.01), loss = 'binary_crossentropy', metrics = 'accuracy')

In [ ]:
image_dir = Path('prediction/testimages')

In [ ]:
file_paths = list(image_dir.glob(r'**/*.*'))

file_paths = pd.Series(file_paths, name='Filepath').astype(str)

test_df = pd.concat([file_paths], axis=1)

In [ ]:
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    # y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    # classes = ['Cat', 'Dog'],
    class_mode=None,
    batch_size=32,
    shuffle=False
)

In [ ]:
test_images.reset()

In [ ]:
predictions = (model.predict(test_images) >= 0.5).astype(np.int)

In [ ]:
filenames = test_images.filenames
file = []
for i in filenames:
    file.append(int(os.path.splitext(os.path.basename(i))[0]))

In [ ]:
result = pd.DataFrame({
    "ImgID":file,
    "Category":predictions[:,0]
})

In [ ]:
result.Category.replace({0:'Cat',1:'Dog'},inplace=True)

In [ ]:
result.sort_values('ImgID',inplace=True)
result.reset_index(drop=True,inplace=True)

In [ ]:
result.to_csv('prediction.csv',index=False)